# Manifold Learning: Introduction and Foundational Algorithms
### *Mathematical Theory with Examples and Applications in Python*
`Drew Wilimitis`

![KleinDual](https://user-images.githubusercontent.com/16658498/55211718-3e737900-51bb-11e9-8a4a-37ecf83c9ab0.gif)
<font size = 2> Image source: Klein's quartic curve by Greg Egan https://www.gregegan.net/SCIENCE/KleinQuartic/KleinQuartic.html <font>

In this series of Jupyter Notebooks, I present the mathematical details of some foundational manifold learning algorithms alongside some applied examples using the `sklearn.manifold` library. I begin with this introduction and provide a very brief, basic overview of some relevant topology and differential geometry, as well as the idea of topological data analysis. I then discuss machine learning with high dimensional datasets and the manifold hypothesis.   
<br>
This project can be viewed with the following links to jupyter notebook viewer. The original jupyter notebook files used to create this project can be downloaded here: https://github.com/drewwilimitis/Manifold-Learning 


Contents
------

* [**Introduction:**](https://nbviewer.jupyter.org/github/drewwilimitis/Manifold-Learning/blob/master/Manifold_Learning_Intro.ipynb)
    - Overview of manifolds and the basic topology of data
    - Statistical learning and instrinsic dimensionality
    - The manifold hypothesis 

* [**Chapter 1: Multidimensional Scaling**](https://nbviewer.jupyter.org/github/drewwilimitis/Manifold-Learning/blob/master/Multidimensional_Scaling.ipynb)
    - Classical, metric, and non-metric MDS algorithms
    - Example applications to quantitative psychology and social science
    
* [**Chapter 2: ISOMAP**](https://nbviewer.jupyter.org/github/drewwilimitis/Manifold-Learning/blob/master/Isomap.ipynb)
    - Geodesic distances and the isometric mapping algorithm
    - Implementation details and application 

* [**Chapter 3: Local Linear Embedding**](https://nbviewer.jupyter.org/github/drewwilimitis/Manifold-Learning/blob/master/Locally_Linear_Embedding.ipynb)
    - Locally linear reconstructions and optimization problems
    - Example applications with image data
    
* [**Chapter 4: Laplacian Eigenmaps/Spectral Embedding**](https://nbviewer.jupyter.org/github/drewwilimitis/Manifold-Learning/blob/master/Laplacian-Eigenmaps.ipynb)
    - From the general to the discrete Laplacian operators
    - Visualizing spectral embedding with the networkx library

## Background: Topology, Metric Spaces, and Manifolds

To get an idea of the theoretical foundation for applied dimensionality reduction, we start very generally with topology and the notion of a *mathematical space*.

A **topological space** is defined only in terms of set theory, and therefore provides a very general way to define a *mathematical space* - which is really just a set with some additional structure. It's not especially necessary to fully understand the following formal definition, but it can certainly be appreciated for its abstraction. A **topological space** is defined as a set $X$ with a collection of open neighborhoods, or subsets, $T$ that follow a few basic axioms: <br>
1. The (trivial) subsets $X$ and the empty set $\emptyset$ are in $T$
2. Whenever sets $A$ and $B$ are in $T,$ then so is $A \cap B$ .
3. Whenever two or more sets are in $T$ , then so is their union

From this definition of a topological space, we add additional structure, such as a metric, that allows us to characterize the relationships between elements of the set. Since notions of similarity and distance, which we formalize as metrics, are central to geometry, machine learning tasks, and especially manifold learning algorithms, we formally define a **metric space**. 

A **metric space** is a pair $(M, d)$ where $M$ is a set and $d$ is a metric on $M$, where a metric is a function $d : M \times M \rightarrow \mathbb{R}$ such that for any $x, y, z \in M$ we have:<br>
<br>
$$
\begin{array}{ll}{d(x, y)=0 \Leftrightarrow x=y} \ \ \ {\text { identity }} \\ {d(x, y)=d(y, x)} \ \ \ {\text { symmetry }} \\ {d(x, z) \leq d(x, y)+d(y, z) \ \ \ \text { triangle inequality }}\end{array}
$$
<br>
The metric function has these rules that we would expect for distances. Just like we can't have negative distance, we also have $d(x, y) \geq 0$:<br>
<br>
$$
\begin{array}{ll}{d(x, y)+d(y, x) \geq d(x, x)} & {\text { by triangle inequality }} \\ {d(x, y)+d(x, y) \geq d(x, x)} & {\text { by symmetry }} \\ {2 d(x, y) \geq 0} & {\text { by identity  }} \\ {d(x, y) \geq 0} & {\text { non-negativity }}\end{array}
$$

![title](https://i.stack.imgur.com/CWRld.png)

This figure shows open unit neighborhoods or 'unit balls' centered at the origin for different metric spaces, where the neighborhoods are defined as the set of elements within a given radius $\varepsilon$: <br>
<br>
$$B_{\varepsilon}(x) :=\{y \in M | d(x, y)<\varepsilon\}$$ <br>
For the unit neighborhood we let $\varepsilon = 1$. From left to right we have the metric spaces $(\mathbb{R^2}, d)$ with the following metrics: <br>
<br>
$$
\begin{array}{ll}{d_2(x, y)=\sqrt{\left(x_{1}-y_{1}\right)^{2}+\left(x_{2}-y_{2}\right)^{2}}} & {\text { the Euclidean metric or } L_2 \text{norm}} \\ {d_{\infty}(x, y)=\max \left(\left|x_{1}-y_{1}\right|,\left|x_{2}-y_{2}\right|\right)} & {\text { maximum metric or } L_{\infty} \text{norm} } \\ {d_1(x, y)=\left|x_{1}-y_{1}\right|+\left|x_{2}-y_{2}\right|} & {\text { taxicab metric or} \ L_{1} \text{norm}  } \end{array}
$$
<br>
Although the Euclidean metric is often standard, there are many cases in machine learning where the specification of the metric is crucially important. Specifying the $L_1$ or $L_2$ norm defines regularization techniques like LASSO and Elastic Net for penalized regression. Many methods in dimensionality reduction and unsupervised learning consider local neighborhoods, which are dependent on the choice of metric, and this is often critical to learning geometric and topological properties.

With the definition of metric spaces, we can continue to build up from general topological spaces and begin to define concepts that are based on distances, like limits, continuous functions, and convergence, eventually getting into more familiar notions like derivatives and integrals from the study of calculus. <br>

Just like metric spaces are just topological spaces with this additional metric structure, a **manifold** is also just a topological space with the additional property that for each point, the surrounding neighborhood locally resembles Euclidean space (which we can take to be the reals $\mathbb{R}$ with the standard Euclidean distance metric). For this reason, manifolds (specifically a special class of manifolds called differential manifolds) allow you to use calculus on higher dimensional structures that might be globally non-Euclidean. 

More formally, each point of an n-dimensional **manifold** has an open neighborhood that is *homeomorphic* to an open neighborhood of Euclidean space with dimension n. Before going further we must define this new term that formalizes the notion of "locally resembling Euclidean space". <br>
<br>
A **homeomorphism** is a function $f : X \rightarrow Y$ between two topological spaces where: <br>
<br>
$$
\begin{array}{l}{\cdot f \text { is a bijection (one-to-one and onto) }} \\ {\cdot f \text { is continuous }} \\ {\cdot \text { the inverse function } f^{-1} \text { is continuous } }\end{array}
$$

A homeomorphism is a continuous bijection (where the inverse is also continuous). We say the two topological spaces are *homeomorphic* if there exists a *homeomorphism* between them. <br>
<br>
**Note**: an n-dimensional manifold is only *locally* homeomorphic to Euclidean space, and in general a manifold is not necessarily globally homeomorphic to Euclidean space. <br>
<br>
**Note**: a manifold can also have additional properties like a metric and differentiability.

Two spaces that are homeomorphic have the same topological properties like compactness and connectedness (the word homeomorphic means "same shape"). Although they are not precisely the same, a homeomorphism can be thought of in a similar way as a continuous deformation - the continuous stretching and bending that preserves topological equivalence. One can often imagine stretching or flattening out local neighborhoods of a manifold to get a flat Euclidean plane. <br>
<br>
Since a homeomorphism is a continuous bijection with a continuous inverse, we can jump back and forth between these topological spaces.

![title](https://upload.wikimedia.org/wikipedia/commons/6/64/Circle_with_overlapping_manifold_charts.svg)<br>
This is a simple demonstration of a manifold (the unit circle in $\mathbb{R}^2$) with four homeomorphic transition maps. For example, the top, yellow part of the circle is mapped continuously into Euclidean space by mapping each point to its x-coordinate: $\phi_{\operatorname{top}}(x, y)=x$, which maps the top part of the circle into the interval (-1,1). The full demonstration with explicitly defined transition maps can be found at the figure source: https://en.wikipedia.org/wiki/Manifold


### 1D Manifolds - Curves

An n-dimensional manifold in a larger m-dimensional space ($n < m$) locally resembles an n-dimensional Euclidean hyperplane. Many simple curves in $\mathbb{R^2}$ are locally homeomorphic to 1-dimensional Euclidean space, and we can imagine some straightforward examples of these 1D manifolds.

The unit circle in $\mathbb{R^2}$ is a 1-dimensional manifold, but a 'figure eight' curve in $\mathbb{R^2}$ is not a 1-dimensional manifold since it is not locally homeomorphic to Euclidean space in the regions with the center crossing point. We have more examples from conics sections: circles, parabolas, hyperbolas, and ellipses.

![title](https://upload.wikimedia.org/wikipedia/commons/thumb/c/cf/Conics_and_cubic.svg/220px-Conics_and_cubic.svg.png) <br>
Figure source: https://en.wikipedia.org/wiki/Manifold

These 1D manifolds don't have to be connected, like the manifold defined as the union of the two circles where the local homeomorphism criteria still holds. Manifolds also don't have to be finite, like the hyperbolas that extend out infinitely. 


### 2D Manifolds - Surfaces

![title](https://user-images.githubusercontent.com/16658498/56078883-99f45800-5db2-11e9-8adb-84f4a5c98aa7.PNG)
Figure source: Renze, John; Rowland, Todd; and Weisstein, Eric W. "Compact Manifold." From MathWorld--A Wolfram Web Resource. http://mathworld.wolfram.com/CompactManifold.html

Many familiar closed surfaces like the sphere, torus, and Klein bottle in $\mathbb{R^3}$ are locally homeomorphic to 2-dimensional Euclidean space, and so these are all 2-dimensional manifolds. You can imagine a tangent plane approximating the local neighborhoods at each point on these surfaces, or flattening out a small patch of the sphere's surface so that it lies along the Euclidean plane in $\mathbb{R^2}$. 

Globally, however, these surfaces are generally not going to be homeomorphic to $\mathbb{R^2}$. You can't find a homeomorphism between the sphere and the Euclidean plane. Intuitively, you can't twist or stretch the sphere continuously to lie flat along the plane (without squishing it to a single point or folding it on top of itself - which are not bijective transformations)


### n-Manifolds

- **Euclidean Space**: $\mathbb{E}^n$ is an n-dimensional manifold, since it is homeomorphic to itself<br>
<br>
- **n-Sphere**: $S^{n}=\left\{x \in \mathbb{R}^{n+1} :\|x\|=r\right\}$ is an n-dimensional manifold<br>
<br>
- **n-Torus**: The n-Torus is an n-dimensional manifold

![title](https://user-images.githubusercontent.com/16658498/59167559-60607480-8af7-11e9-9b61-4dc4abe13b80.png)<br>
The continuous surface of a 3-D bunny (which is modeled here by a discrete mesh) can also be defined as a manifold, and calculus can be applied to this non-Euclidean shape. Source: http://graphics.stanford.edu/data/3Dscanrep/ 

<div class="alert alert-block alert-info">
<b>Big Idea of Manifolds</b> <br>
    <br>
    Since a manifold structure is defined by this property of *locally* resembling Euclidean space, we do not have to consider the geometric relationships of any global, extrinsically defined coordinate system and instead we can just consider the manifold's intrinsic geometry and topological properties.
</div>

### The Idea Behind Topological Data Analysis

Geometric and topological relationships are fundamental to essentially every data analysis and machine learning task, as we use geometry to identify similarities and distinctive characteristics in the data. In classification, for example, data points that are similar (close to each other) are assigned to the same classes and data points that are significantly different (i.e. far apart in one or many of the feature dimensions) are assigned to different labels. <br>
<br>
We usually consider data as a finite set $X=\left\{x_{1}, x_{2}, \cdots, x_{N}\right\} \subset \mathbb{R}^{D}$ sometimes called a point cloud. However, geometric and topological structures like metric spaces and manifolds are continuous, not discrete. To discover any geometric or topological properties of the data, we \textbf{fit a continuous shape to the data}, and we must make certain assumptions about the underlying mathematical space we're working in. It is often simply assumed that our data lies in the standard Euclidean space with the typical Euclidean metric, and the data is analyzed by referencing a global, external coordinate system. However, many interesting and important structures that arise are actually non-Euclidean, and by fitting a continuous shape (such as a manifold) to the data, we can translate our data analysis task from an external, global coordinate system (possibly having very high dimensionality) into the intrinsic coordinate system defined by the assumed manifold structure itself. Ideally, the underlying manifold will capture the latent structure in our dataset.

In [1]:
# ![title](https://raw.githubusercontent.com/drewwilimitis/Manifold-Learning/master/images/topological-data-analysis-15-638.jpg) <br>
Figure source: https://www.nature.com/articles/srep01236#author-information

SyntaxError: invalid syntax (<ipython-input-1-1023b59114ce>, line 2)

# Once we assume that the data can be approximated by a manifold, then we can use the fact that topological properties remain constant with continuous deformations. We can often apply a dimensionality reduction transformation to map the data into lower dimensional spaces, while preserving important topological properties (such as connectedness and compactness). For this geometric and topological information to be useful, the assumed, underlying continuous shape needs to reveal important features of the data and the noise of the data samples needs to be relatively low.<br>
<br>
To further understand the utility of topological approaches and how they can uncover highly complex and informative features (to be used for visualization, dimensionality reduction, and/or as engineered features to be input to a supervised learning model) we must first consider the challenges of analyzing high dimensional data. 

## The Curse (and Blessing) of Dimensionality

> **The Curse of Dimensionality** refers to the various challenges (such as computational complexity) that arise when applying data analysis and machine learning methods to high dimensional datasets.

As the dimensionality of a dataset increases, the volume of the high dimensional space grows exponentially. The available data can become very sparse in relation to higher dimensional volumes, where essentially all the data points become further and further apart. This makes machine learning tasks that rely on identifying feature interactions and similarities in groups of data, like clustering, extremely hard and often computationally intractable. 

### <font color='blue'> Example - Sampling uniformly within n-dimensional unit spheres </font>

We first import some helpful libraries and configure some output settings.

In [4]:
# import libraries
import numpy as np
import math
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.core.pylabtools import figsize
plt.style.use('default')
plt.rcParams['figure.facecolor'] = 'white'
figsize(12, 6)

# display multiple outputs within a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all";

# ignore warnings
import warnings
warnings.filterwarnings('ignore');


Bad key text.latex.unicode in file /home/deniz/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file /home/deniz/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file /home/deniz/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably need to get an updated matplotlibrc file from
https://github.com/

Consider data sampled uniformly within the unit sphere of dimension $n$: <br> 
<br>
$$\mathbf{B}^{n}=\left\{\left(x_{1}, x_{2}, \ldots, x_{N}\right) \in \mathbb{R}^{n} | x_{1}^{2}+x_{2}^{2}+\cdots+x_{N}^{2}<1\right\}$$

Below we sample uniformly within the 2 and 3-dimensional unit spheres. We consider neighborhoods centered at the origin and the proportion of samples that lie within the neighborhood.

In [ ]:
# plot unit circle in R^2
fig = plt.figure()
ax = fig.add_subplot(1, 2, 1)
circ = plt.Circle((0, 0), radius=1, edgecolor='black', facecolor='None', linewidth=3, alpha=0.5)
ax.add_patch(circ)
circ = plt.Circle((0, 0), radius=0.5, edgecolor='black', facecolor='None', linewidth=3, linestyle='--')
ax.add_patch(circ)

# sample within unit sphere in R^2
n = 1000
theta = np.random.uniform(0, 2*math.pi, n)
u = np.random.uniform(0, 1, n)
r = np.sqrt(u)
x = r * np.cos(theta)
y = r * np.sin(theta)
ax.scatter(x, y, s=10, alpha=1)

# proportion within neighborhood
p = np.sum((x**2 + y**2) <= 0.5**2) / n
ax.scatter(0, 0, s=20, c='black')
plt.title("% in neighborhood = " + str(p))

# plot unit sphere in R^3
ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.set_aspect("auto")
u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
x = np.cos(u)*np.sin(v)
y = np.sin(u)*np.sin(v)
z = np.cos(v)
ax.plot_wireframe(x, y, z, color="black", alpha=0.5)
ax.plot_wireframe(0.5*x, 0.5*y, 0.5*z, color="black")

# uniform sampling
phi = np.random.uniform(0, 2*math.pi, n)
costheta = np.random.uniform(-1, 1, n)
u = np.random.uniform(0, 1, n)

theta = np.arccos(costheta)
r = u**(1/3)
x = r * np.sin(theta) * np.cos(phi)
y = r * np.sin(theta) * np.sin(phi)
z = r * np.cos(theta)
ax.scatter(x, y, z, alpha=0.4, s=10)

# proportion within neighborhood
p = np.sum((x**2 + y**2 + z**2) <= 0.5**2) / n
ax.scatter([0], [0], [0], color="black", s=100)
plt.title("% in neighborhood = " + str(p));

In $\mathbb{R^2}$ the unit sphere has volume $V = \pi \cdot r^2$

If we let p = % of samples within the neighborhood, we have p approximately equal to the volume of the neighborhood divided by the volume of the unit sphere. For $r = \frac{1}{2}$ we have $p \approx (\frac{1}{2})^2 = \frac{1}{4}$

In $\mathbb{R^3}$ the unit sphere has volume $V = \frac{4}{3} \pi \cdot r^3$ and so for $r = \frac{1}{2}$ we have $p = (\frac{1}{2})^3 \approx \frac{1}{8}$

In $\mathbb{R^n}$, the unit sphere has volume that increases exponentially with a given fixed radius:
$$
V_{n}(R)=\frac{\pi^{\frac{n}{2}}}{\Gamma\left(\frac{n}{2}+1\right)} R^{n}
$$

where in the denominator we have the Gamma function: <br>
$$\Gamma(n)=(n-1) ! \text { if } n \text { is a positive integer }$$

In 10 dimensions we have the volume $$
V = \frac{\pi^{5}}{120} R^{10} \approx 2.550 \times R^{10}
$$

and so if the radius is 1/2, the proportion of samples within our neighborhood $p \approx 2.43^{-3}$

To have the neighborhood contain approximately 1/4 of the samples like the sphere in $\mathbb{R^2}$ we need to have $r \approx 0.8$ which is nearly the size of the whole dataset. Clustering tasks become nearly impossible when the data points are so far away and near the boundary, and the volume of the high dimensional space becomes so enormous. We would need training samples that grow exponentially with the dimensionality, which quickly becomes computationally intractable.

There are many methods in machine learning that attempt to reduce the dimensionality of high dimensional datasets. In some sense, they all rely on exploiting the intrinsic structure of the data, where the dataset lies closely to a lower dimensional subspace embedded within the higher dimensional space. Empirically, high dimensional data sets are not sampled uniformly, and so there are special structures and irregularities like many data points being very close together in one of the dimensions. 

This fact that higher dimensional datasets often have some intrinsic lower dimensionality is what I refer to as this blessing of dimensionality. 

### <font color='blue'> Example - MNIST Digits </font>

Many datasets are actually very very far from being sampled uniformly over some domain. Consider a classification problem with the MNIST handwritten digits dataset.

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
mnist.data.shape

(70000, 784)

We have 70000 handwritten digits 0-9, where each observation is an image with $28 \times 28 = 784$ features that can take values representing pixel intensities from 0 to 255. Here is a plot of some examples:

In [3]:
fig, ax = plt.subplots(6, 8, subplot_kw=dict(xticks=[], yticks=[]))
for i, axi in enumerate(ax.flat):
    jj = axi.imshow(mnist.data.iloc[1250 * i].values.reshape(28, 28), cmap='gray');

NameError: name 'plt' is not defined

If you imagine generating random images by sampling uniformly over this 784 dimensional space, the probability that you would generate an image resembling a handwritten digit is unbelievably small. There's some intrinsic structure to this dataset that could allow for dimensionality reduction, such as all the digits being centered and the corners being almost all white, common line segments and loops, and a bimodal distribution of pixel values where they are usually either black or white.  

In [ ]:
# explained variance from principal components
from sklearn.decomposition import PCA
# take subset of mnist data (1/10 of observations)
data = mnist.data[::10]
model = PCA()
proj = model.fit_transform(data)
cumsum = np.cumsum(model.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1
plt.figure(figsize=(8,5))
plt.plot((0, 800), (0.95, 0.95), 'r--')
plt.plot(cumsum)
plt.xlim(0,200)
plt.style.use('seaborn')
plt.title('PCA Explained Variance')
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()
print('95% Explained Variance with ' + str(d) + ' Principal Components');

The original data is 28 x 28 = 784 dimensions, but with only the first ~150 principal component features we can preserve 95% of the variation among the handwritten digits data. Therefore, it's possible to use the principal components and project the data into this 150 dimensional space without too much information loss, as the remaining 634 dimensions are relatively uninformative. In this sense we can consider the intrinsic dimensionality of the mnist data as much lower than the extrinsic 784 dimensional space.

If we have to predict the target class of a new validation sample, we know a lot already about the 784 feature values simply because we know we're predicting handwritten digits in this problem. This is the case with many other high dimensional datasets as well. For example, in an image classification task for a dataset with images of animals, samples will have discernible objects and probably some common structures like eyes, ears, tails, etc.   


## The Manifold Hypothesis

Since many real datasets lie close to some lower dimensional subspace, many techniques for dimensionality reduction are also based on assuming a lower dimensional manifold representation. 

> The **manifold hypothesis**, or **manifold assumption**, is that many real-world datasets can be approximately represented as lower dimensional manifolds that are embedded in a higher dimensional space.

**Manifold learning** is a class of methods developed to learn this lower dimensional representation, sometimes referred to as the intrinsic dimensionality, of the data. <br>
<br>
In manifold learning, the data points $x_1, x_2, ..., x_N$ are assumed to be sampled from the (often uniform) distribution defined by the underlying d-dimensional manifold $M \subset \mathbb{R}^{D}$. We then attempt to learn the intrinsic structure of this underlying manifold. This can all be quite abstract, but below is a more concrete example with the canonical manifold learning "swiss roll" dataset, a 3-dimensional dataset that lies close to a nonlinear, 2-dimensional manifold.

In [ ]:
# draw samples to create the grid
t = np.linspace(0, 1, 50)
u = np.linspace(0, 1, 50)
v = 3*np.pi/2*(.1 + 2*t)
u,v = np.meshgrid(u,v)

# swiss roll transformation
x = -v*np.cos(v)
y = u
z = v*np.sin(v)

# set up a figure twice as wide as it is tall
fig = plt.figure(figsize=plt.figaspect(0.5))
plt.style.use('default')
plt.rcParams['figure.facecolor'] = 'white'

# plot 3d surface
colors = plt.cm.jet((x**2 + z**2) / 100)
ax = fig.add_subplot(1, 2, 1, projection='3d')
ax.plot_surface(x, y, z, rstride=1, cstride=1, facecolors=colors, cmap=plt.cm.coolwarm, linewidth=1.4, alpha=0.8)
ax.set_title('Continuous manifold in $\mathbb{R}^3$', size=16);

# draw uniform samples from the continuous manifold
n = 4000
t = np.random.rand(n, 1)
u = np.random.rand(n, 1)
v = 3*np.pi/2*(.1 + 2*t)

x = -v*np.cos(v)
y = u
z = v*np.sin(v)
color = (x**2 + z**2) / 100
color = color.reshape(n,)

# set up the axes for the first plot
ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.scatter(x, y, z, cmap=plt.cm.jet, c=color);
ax.set_title('Finite Data Samples', size=16);
plt.show();

Statistical considerations must be taken into account when we try to uncover the true latent structure of our dataset. When we assume the data lies close to this embedded manifold, we view the collection of data points we have to be a finite sample drawn from the continuous distribution of the manifold. <br>
<br>
Therefore, statistical methods can be applied to estimate the true topological features of the manifold based on our observed, finite data samples. The bias and variance can be assessed and it's possible to determine whether the noise in our dataset allows us to derive statistically significant claims about the assumed manifold geometry.   

Two related problems that arise are significant levels of noise in the input data and insufficient sampling that under-represents important regions. Consider how introducing a little noise seems to obfuscate the latent manifold geometry (this time using the `sklearn.manifold` library to generate the swiss roll).

In [ ]:
# set up a figure twice as wide as it is tall
fig = plt.figure(figsize=plt.figaspect(0.5))

# sample from swiss roll dataset
from sklearn.datasets.samples_generator import make_swiss_roll
X, color = make_swiss_roll(n_samples = 1000, noise=1.5)
ax = fig.add_subplot(1, 2, 1, projection = '3d')
ax.scatter(X[:, 0], X[:, 1], -X[:, 2], c=color, cmap=plt.cm.jet)
ax.set_title('Sampling with Gaussian noise', size=16)

# undersample from manifold
X, color = make_swiss_roll(n_samples = 100, random_state=8888)
ax = fig.add_subplot(1, 2, 2, projection = '3d')
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=color, cmap=plt.cm.jet)
ax.set_title('Undersampling', size=16);
plt.show();

Manifold learning is intriguing and insightful in its own right and it offers a valuable way to understand and visualize latent geometries and invariant structural relationships. However, an implicit assumption in most of these methods is that they will assist supervised learning problems. Manifold learning is often incorporated in supervised learning problems as an exploratory analysis/visualization, feature extraction method, and for non-linear dimensionality reduction.<br>
<br>
In this project, I don't go over the details of Principal Component Analysis (PCA), except for a few cases where PCA naturally arises as a point of comparison to the other algorithms. There are many many great resources on PCA, and including an in-depth treatment of PCA would significantly expand the scope of this project. None of the material is dependent on PCA, but there is one crucial thing to remember about PCA in the broader context of manifold learning and dimensionality reduction.<br>
<br>
PCA involves a *linear* projection of the original data onto a selected number of principal component vectors (vectors that maximize the variance in the data). Therefore, standard methods from PCA are unable to learn *non-linear* relationships, whereas the class of algorithms in manifold learning are able to learn non-linear relationships and preserve these non-linear relationships when transforming the data to lower dimensions. <br>
<br>
It can be hard to know whether non-linear dimensionality reduction will simplify a machine learning task, and this isn't always the case, but often times classification and regression problems are easier after applying lower dimensional embeddings. Below is an example of manifold learning simplifying a basic classification problem, while PCA fails to learn the non-linear structure.

In [ ]:
X, color = make_swiss_roll(n_samples = 3000)
plt.style.use('classic')
plt.rcParams['figure.facecolor'] = 'white'
fig = plt.figure(figsize = (12, 6))
ax = fig.add_subplot(1, 2, 1, projection = '3d')
ax.scatter(X[(color < 10), 0], X[(color < 10), 1], X[(color < 10), 2], c='blue')
ax.scatter(X[(color >= 10), 0], X[(color >= 10), 1], X[(color >= 10), 2], c='red')
ax.set_title('Swiss Roll', size=16)

ax = fig.add_subplot(1, 2, 2, projection = '3d')
ax.scatter(X[(color < 10), 0], X[(color < 10), 1], X[(color < 10), 2], c='blue')
ax.scatter(X[(color >= 10), 0], X[(color >= 10), 1], X[(color >= 10), 2], c='red')
ax.set_title('Alternate View', size=16)
ax.view_init(4, -80);
plt.show();

Notice that in $\mathbb{R^3}$ it would be hard to fit a linear decision boundary and classify data points. After applying the Isomap manifold learning algorithm (covered in Part 2 of this manifold learning series) to reduce the dimensionality to $\mathbb{R^2}$ and visualize the intrinsic dimensionality, the classification task becomes much easier. The Isomap algorithm is able to learn the shape of the data, while PCA fails, and the geometry provides the key insight into the similarities between data points.  

In [ ]:
from sklearn.manifold import Isomap
from sklearn.decomposition import PCA

fig = plt.figure(figsize=(12,6))
data = X
k = 8

pca = PCA(n_components=2)
pca.fit(data)
X_pca = pca.transform(data)

ax = fig.add_subplot(1, 2, 1)
ax.scatter(X_pca[(color < 10), 0], X_pca[(color < 10), 1], c='blue')
ax.scatter(X_pca[(color >= 10), 0], X_pca[(color >= 10), 1], c='red')
ax.set_title('PCA with two components', size=14)

model = Isomap(n_components=2, n_neighbors=k)
X_iso = model.fit_transform(data)
ax = fig.add_subplot(1, 2, 2)
ax.scatter(X_iso[(color < 10), 0], X_iso[(color < 10), 1], c='blue')
ax.scatter(X_iso[(color >= 10), 0], X_iso[(color >= 10), 1], c='red')
ax.set_title('Isomap with k = ' + str(k), size=14);

Using the nonlinear Isomap transformation, we can easily fit a linear decision boundary to separate the classes.

In [ ]:
from mlxtend.plotting import plot_decision_regions
from sklearn.svm import LinearSVC
clf = LinearSVC()
plt.style.use('default')
y = np.zeros((len(X_iso)))
y[color < 10] = 0
y[color >= 10] = 1
y = y.astype('int')
svm_clf = clf.fit(X_iso, y)
plt.figure(figsize=(10,5.5))
ax = plot_decision_regions(X_iso, y, clf=svm_clf, legend=2, markers='oo', colors='blue,red')
# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), shadow=True)
plt.show();

The non-linear dimensionality reduction has greatly simplified the classification task, and this example demonstrates a simpler case when manifold learning can be used as a pre-processing step to improve supervised learning models. <br>
<br>
There are a few broad classes of these manifold learning algorithms, and the recurring themes provide an excellent entry point to explore some deeper mathematical topics like topological methods in machine learning and data analysis, which have a wide variety of applications, many of them probably yet to be discovered. There are well-developed implementations of many of these algorithms and some existing demonstrations of their effectiveness on a variety of interesting problems. Throughout this project, I go over some of these non-linear dimensionality reduction algorithms and manifold learning methods in more detail. <br>

In [ ]:
from IPython.core.display import HTML

def css_styling():
    styles = open("custom_style.css", "r").read()
    return HTML(styles)
css_styling()

___

### References


-  [1] https://en.wikipedia.org/wiki/Manifold
-  [2] https://en.wikipedia.org/wiki/Metric_space
-  [3] https://en.wikipedia.org/wiki/Topological_space